# Prototype of lipreading pipeline

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import random
import math
import time

# model file, encoder, decoder and seqtoseq
from model import *
# utils file
from utils import *
# Get landmark using vocadataset.py
from data.vocaset import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Get landmark from vocadaset class
#trainset = vocadataset("train", landmark=True)
trainset = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

trainloader = DataLoader(trainset, batch_size=2, collate_fn=collate_fn)

landmarks, len_landmark, label, len_label = next(iter(trainloader))


In [3]:
vocabulary = create_vocabulary(blank="@")

In [4]:
# Create a mapping from characters to indices
char_to_index = {char: index for index, char in enumerate(vocabulary)}

In [5]:
# Convert the sequence and target to indices
#sequence_indices = [char_to_index[char] for char in sequence]

label_t = char_to_index_batch(label, vocabulary)

#target_indices = [char_to_index[char] for char in labels]
#target_tensor = torch.tensor(target_indices)

In [9]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 256
output_dim = len(vocabulary)

model = only_Decoder(INPUT_DIM, HID_DIM, 8, len(vocabulary)).to(device)




/home/hsilva/miniconda3/envs/env/lib/python3.10/site-packages/torch/cuda/__init__.py:123: UserWarning: 
    Found GPU1 NVIDIA GeForce GTX TITAN which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))


In [ ]:
landmarks.shape

In [1]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss(blank=1)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    
    for i in range(landmarks.shape[0]):
        landmarks[i] = landmarks[i].to(device) 
        label_t[i] = label_t[i].to(device)
        optimizer.zero_grad()
        target_tensor = label_t[i]
        reshaped_landmark = torch.reshape(landmarks[i], (landmarks[i].shape[0], landmarks[i].shape[1]*landmarks[i].shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        output = model(reshaped_landmark[None, : , :], target_tensor[None, None,:])

        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.long)
        
        loss = ctc_loss(output, target_tensor, input_lengths, target_lengths[0])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



NameError: name 'nn' is not defined

In [ ]:
target_tensor.shape

In [ ]:
# Decode the output sequence
output_indices = torch.argmax(output, dim=2).squeeze(1)
output_sequence = ''.join([vocabulary[index] for index in output_indices])

print("Target Sequence:", labels.replace("@","").replace("#",""))
print("Decoded Output:", process_string(output_sequence))